# Importing libraries

In [ ]:
import os, glob, csv, subprocess, sys, re
from urllib.request import urlopen
import urllib.request
from bs4 import BeautifulSoup
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd

# Defining directories

In [ ]:
project = ""        # put the cloned repository name in the project, e.g. activemq, hbase, etc.
bugidentifier = ""  # put the bug identifier from the project, e.g. AMQ for activemq project, HBASE for hbase project, etc.
userhome = os.path.expanduser('~')
repository = userhome + r'/DifferentDiffAlgorithms/SZZ/datasource/' + project + '/'
analyze_dir = userhome + r'/DifferentDiffAlgorithms/SZZ/projects_analyses/' + project + '/'

# Defining git command function

In [ ]:
def execute_command(cmd, work_dir):
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Finding log messages containing bug

Defining the keywords

In [ ]:
words = ['bug', 'fix', 'defect', 'patch']

In [ ]:
#Applying git command to find commit log messages containing words of bugs, fix, defect or patch
logs = []
for n, word in enumerate(words):
    git_cmd = "git log --all --grep='" + word + "' --oneline"
    log = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('b"','').replace('(','',1).replace("\\'","'").split("\\n"))[:-1]
    logs.append(log)

com_logs = [item for sublist in logs for item in sublist]
com_logs

In [ ]:
#separating the commit id and the log messages
commit = []
for xx in range(0,len(com_logs)):
    tmp = []
    comm = com_logs[xx].split()
    word = ' '.join(comm[1:])
    tmp.extend([comm[0],word])
    commit.append(tmp)

for item in commit:
    print (item)

# Capturing candidate bug-fix commits

In [ ]:
bugfix_commit = []
for t in range(0,len(commit)):
    com = commit[t][0]
    try:
        bugid = re.search("(" + bugidentifier + "-[0-9]+)", commit[t][1])
        bugid = bugid.groups()[0]
        bugfix_commit.append([com, bugid])
    except:
        pass

print ("Number of commit logs = %i" % len(bugfix_commit))

In [ ]:
# Capturing only bug id candidates which are identified as bug links in the log messages 
# to synchronize the type of bug id from JIRA ITS
commitbugs = []
buglinks = []

for xx in range(0,len(bugfix_commit)):
    if bugfix_commit[xx][1] not in buglinks:
        buglinks.append(bugfix_commit[xx][1])
    else:
        pass

buglinks = sorted(buglinks, reverse=True)
print ("\nNumber of bug id: " + str(len(buglinks)))
buglinks

# Synchronizing the bug ids with the bug report from JIRA ITS database

In [ ]:
#Synchronizing the bug ids with the bug report from JIRA ITS database
errorlinks = []
id_type = []
for a,b in enumerate(buglinks):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " %(a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        resolution = soup.find('span', attrs={'id':'resolution-val'}).text.replace("\n",'').replace(" ",'').split(",")
        types = soup.find('span', attrs={'id':'type-val'}).text.replace("\n",'').replace(" ",'').split(",")
        types = sorted(types)
        types.insert(0, b)
        types.insert(2,resolution[0])
        id_type.append(types)
    except:
        errorlinks.append(b)

print("\nExtraction has been completed.")

In [ ]:
#Finding out if there were any uncollected type of bug ids from the sync process
type_of_id = id_type
errorlinks

In [ ]:
#Resync for the error bug ids identified
#This program should be run multiple times if there are still some unscrapped bug ids due to network problems
error_links = []
for a,b in enumerate(errorlinks):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " %(a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        resolution = soup.find('span', attrs={'id':'resolution-val'}).text.replace("\n",'').replace(" ",'').split(",")
        types = soup.find('span', attrs={'id':'type-val'}).text.replace("\n",'').replace(" ",'').split(",")
        types = sorted(types)
        types.insert(0, b)
        types.insert(2,resolution[0])
        id_type.append(types)
    except:
        error_links.append(b)

print("\nExtraction has been completed.")

In [ ]:
type_of_id.sort()
error_links

In [ ]:
bug_links = []
for d, types in enumerate (type_of_id):
    if types[1] == 'Bug' and types[2] == 'Fixed':
        bug_links.append(types[0])

print ("Number of bug id: " + str(len(bug_links)))
print (bug_links)

In [ ]:
with open (analyze_dir + "01_bug_ids_extraction/candidate_bug_ids.txt", mode="wt", encoding="utf-8") as myfile:
    myfile.write('\n'.join(bug_links))
print ("File candidate_bug_links.txt has been created")

# Extracting parent_id of bug-fix commits

In [ ]:
parent_id = []
for v, w in enumerate(bugfix_commit):
    sys.stdout.write("\rExtracting parent id from bug-fix commit: {} / {}".format((v+1), len(bugfix_commit)))
    sys.stdout.flush()
    
    if w[1] in bug_links:
        git_cmd = "git log --pretty=%P -n1 " + w[0]
        temp = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('(','',1).split("\\n"))[:-1]
        dt = [w[1],w[0],temp[0]]
        parent_id.append(dt)
print ("\nFinished")

In [ ]:
parent_id

Saving parent id in CSV file

In [ ]:
with open(analyze_dir + "01_bug_ids_extraction/parentid_of_bugfixcommit.csv", 'w') as parent:
    header = ['bug_id','bugfix_commitID','parent_id']
    writers = csv.writer(parent, delimiter=',')
    writers.writerow(header)
    for item in parent_id:
        writers.writerow(item)

dfparent = pd.read_csv(analyze_dir + "01_bug_ids_extraction/parentid_of_bugfixcommit.csv")
dfparent = dfparent[header]
dfparent

# Saving commit_id of candidate bug fix commits

In [ ]:
commitid_fix = dfparent[['bug_id','bugfix_commitID']]
commitid_fix.to_csv(analyze_dir + "01_bug_ids_extraction/candidatebugfix_commitid.csv", index=False)
print ("File commitid_of_candidatebugfix.csv has been created")